In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phomt-dataset/PhoMT/README.md
/kaggle/input/phomt-dataset/PhoMT/detokenization/test/test.vi
/kaggle/input/phomt-dataset/PhoMT/detokenization/test/test.en
/kaggle/input/phomt-dataset/PhoMT/detokenization/train/train.vi
/kaggle/input/phomt-dataset/PhoMT/detokenization/train/train.en
/kaggle/input/phomt-dataset/PhoMT/detokenization/dev/dev.en
/kaggle/input/phomt-dataset/PhoMT/detokenization/dev/dev.vi
/kaggle/input/phomt-dataset/PhoMT/tokenization/test/test.vi
/kaggle/input/phomt-dataset/PhoMT/tokenization/test/test.en
/kaggle/input/phomt-dataset/PhoMT/tokenization/train/train.vi
/kaggle/input/phomt-dataset/PhoMT/tokenization/train/train.en
/kaggle/input/phomt-dataset/PhoMT/tokenization/dev/dev.en
/kaggle/input/phomt-dataset/PhoMT/tokenization/dev/dev.vi


In [2]:
from typing import List, Optional, Set
from sklearn.base import BaseEstimator, TransformerMixin
import fasttext
from transformers import MarianTokenizer, MarianMTModel
import os
import requests
#from marianmt.preprocess import MarianTokenizer

In [3]:
from transformers import MarianTokenizer, AutoModelForSeq2SeqLM
text ='Có rất nhiều khó khăn.'
mname = 'Helsinki-NLP/opus-mt-vi-en'
tokenizer = MarianTokenizer.from_pretrained(mname)
model = AutoModelForSeq2SeqLM.from_pretrained(mname)
input_ids = tokenizer.encode(text, return_tensors="pt")
outputs = model.generate(input_ids)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded) # Câu chuyện bắt đầu với buổi lễ đếm ngược.

/opt/conda/lib/python3.7/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1292: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


There's a lot of trouble.


In [54]:
#import re
#with open('/kaggle/input/phomt-dataset/PhoMT/tokenization/test/test.en', 'r') as file:
 #   my_string = file.read().replace('\n', ' ')


In [55]:
#my_string = str(my_string)
#my_string = my_string.replace(r"\'", "'")


In [116]:
import nltk

# Load the English sentence tokenizer
nltk.download('punkt')
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
###
# Read the train.en file and split into sentences
with open('/kaggle/input/phomt-dataset/PhoMT/tokenization/test/test.en', 'r', encoding='utf-8') as f:
    text = f.read()
sentences = text.splitlines()

# Print the list of sentences
#print(sentences)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
#len(sentences)

19151

In [60]:
with open('/kaggle/input/phomt-dataset/PhoMT/tokenization/test/test.vi', 'r', encoding='utf-8') as f3:
    text3 = f3.read()
sentences_vi_origin = text3.splitlines()

In [61]:
#len(sentences_vi_origin)

19151

Find Bleu Score

In [65]:
translated_en = []

In [66]:
for v in sentences_vi_origin:
    inputs1 = tokenizer(v, return_tensors="pt")
    outputs1 = model.generate(**inputs1)
    decoded1 = tokenizer.decode(outputs1[0], skip_special_tokens=True)
    translated_en.append(decoded1)

In [70]:
sentences[2:5]

['Two days of heavy rain , high winds , and numerous tornadoes caused major damage across multiple states .',
 'Sadly , Brother Albert Barnett and his wife , Sister Susan Barnett , 85 and 75 years old respectively , were killed when a tornado struck their mobile home .',
 "The United States branch also reports that at least four of our brothers ' homes sustained minor damage , along with two Kingdom Halls ."]

In [76]:
with open("translated_en.txt", "w") as f5:
    for sentence in translated_en:
        f5.write(sentence + "\n")

In [68]:
translated_en[2:5]

['Big rain and heavy wind for two days, along with a lot of tornadoes, caused serious damage to many states.',
 'Sadly, Albert Barnett was 85 years old, and his wife, 75 - year - old Susan Barnett, was killed by a tornado that swept through their home.',
 'The United States branch also reports that at least four homes of our brothers and two Kingdom Halls suffered minor damage.']

In [78]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
ref_tokenized = [nltk.word_tokenize(sentence.lower()) for sentence in sentences]
trans_tokenized = [nltk.word_tokenize(sentence.lower()) for sentence in translated_en]

# Define smoothing function
smooth_func = SmoothingFunction().method1

# Compute BLEU score with smoothing
weights = (0.25, 0.25, 0.25, 0.25)
score = nltk.translate.bleu_score.corpus_bleu(ref_tokenized, trans_tokenized,
                                              weights=weights, smoothing_function=smooth_func)

print(score)

3.3589694639786826e-05


In [81]:
[tokenized_sentences]

[[['Brother',
   'Albert',
   'Barnett',
   'and',
   'his',
   'wife',
   ',',
   'Sister',
   'Susan',
   'Barnett',
   ',',
   'from',
   'the',
   'West',
   'Congregation',
   'in',
   'Tuscaloosa',
   ',',
   'Alabama'],
  ['Severe',
   'storms',
   'ripped',
   'through',
   'parts',
   'of',
   'the',
   'southern',
   'and',
   'midwestern',
   'United',
   'States',
   'on',
   'January',
   '11',
   'and',
   '12',
   ',',
   '2020',
   '.'],
  ['Two',
   'days',
   'of',
   'heavy',
   'rain',
   ',',
   'high',
   'winds',
   ',',
   'and',
   'numerous',
   'tornadoes',
   'caused',
   'major',
   'damage',
   'across',
   'multiple',
   'states',
   '.'],
  ['Sadly',
   ',',
   'Brother',
   'Albert',
   'Barnett',
   'and',
   'his',
   'wife',
   ',',
   'Sister',
   'Susan',
   'Barnett',
   ',',
   '85',
   'and',
   '75',
   'years',
   'old',
   'respectively',
   ',',
   'were',
   'killed',
   'when',
   'a',
   'tornado',
   'struck',
   'their',
   'mobile',
  

In [73]:
tokenized_translated_en

[['Albert',
  'Barnett',
  'and',
  'Susan',
  'Barnett,',
  'members',
  'of',
  'the',
  'West',
  'Congregation',
  'in',
  'Tuscoloosa,',
  'Alabama'],
 ['On',
  'November',
  '11',
  'and',
  'December',
  '12,',
  '2020,',
  'major',
  'storms',
  'swept',
  'through',
  'and',
  'destroyed',
  'parts',
  'of',
  'the',
  'south',
  'and',
  'central',
  'United',
  'States.'],
 ['Big',
  'rain',
  'and',
  'heavy',
  'wind',
  'for',
  'two',
  'days,',
  'along',
  'with',
  'a',
  'lot',
  'of',
  'tornadoes,',
  'caused',
  'serious',
  'damage',
  'to',
  'many',
  'states.'],
 ['Sadly,',
  'Albert',
  'Barnett',
  'was',
  '85',
  'years',
  'old,',
  'and',
  'his',
  'wife,',
  '75',
  '-',
  'year',
  '-',
  'old',
  'Susan',
  'Barnett,',
  'was',
  'killed',
  'by',
  'a',
  'tornado',
  'that',
  'swept',
  'through',
  'their',
  'home.'],
 ['The',
  'United',
  'States',
  'branch',
  'also',
  'reports',
  'that',
  'at',
  'least',
  'four',
  'homes',
  'of',
  '

In [85]:
pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [90]:
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity

In [ ]:
model = Word2Vec([sentence.split() for sentence in corpus1 + corpus2], min_count=1)

# Compute the WMD between the two corpora
instance = WmdSimilarity(corpus1, model, num_best=1)
wmd_scores = instance[corpus2]

# Print the WMD scores for each pair of sentences
for i, score in enumerate(wmd_scores):
    print("WMD Score for Sentence Pair {}: {:.4f}".format(i+1, score[0]))

In [132]:
from bert_score import score
P, R, F1 = score(translated_en, sentences, lang='en', model_type='bert-base-uncased', verbose=True)

print(f'Precision: {P.mean().item():.6f}')
print(f'Recall: {R.mean().item():.6f}')
print(f'F1 Score: {F1.mean().item():.6f}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/597 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/300 [00:00<?, ?it/s]

done in 1392.89 seconds, 13.75 sentences/sec
Precision: 0.788241
Recall: 0.770972
F1 Score: 0.778374


In [131]:
tokenized_sentences2 = [sentence.split() for sentence in sentences[:639]]
tokenized_translated_en2 = [sentence.split() for sentence in translated_en[:639]]

bleu2 = corpus_bleu(tokenized_sentences2, tokenized_translated_en2)
bleu2

0.39292432971921326

In [120]:
sentences[644]

'It took building a whole bunch of systems to get it right , but once we did , we were able to cut our carbon footprint nearly in half .'

In [125]:
translated_en[644]

'So the window farm has grown through a rapid process of transformation similar to software.'

In [135]:
translated_en[34:40]

['- Mrs. Forresta Bakoyannis and Mrs. Renalde Maull Pasquier, two members of the Committee of the Council of Congress of the European Committee for the Russian Federation.',
 "Russia's lack of respect for religious freedom is an inexcusable act of violation of OSCE's commitment to Moscow.",
 'Those who exercise faith in a mild manner should not be persecuted, punished, or imprisoned.',
 'The court’s order to seize the property of Jehovah’s Witnesses only adds to the damage done to the Witnesses.',
 'I hope this case goes to the European Court of Human Rights.',
 '- Senator Roger Wicker, chairman of the Committee for security and cooperation in Europe.']

In [140]:

# define your reference and candidate sentences
referencess = [['the', 'cat', 'is', 'on', 'the', 'mat'], ['there', 'is', 'a', 'cat', 'on', 'the', 'mat']]
candidatess = [['the', 'cat', 'is', 'on', 'the', 'mat'], ['there', 'is', 'a', 'cat', 'on', 'the', 'mat']]

# compute the corpus-level BLEU score
scores = corpus_bleu(referencess, candidatess)

print("Corpus-level BLEU score:", scores)


Corpus-level BLEU score: 0.5266403878479265


In [155]:
jk =[tokenized_sentences]
#jk

In [159]:
tokenized_translated_en

[['Albert',
  'Barnett',
  'and',
  'Susan',
  'Barnett,',
  'members',
  'of',
  'the',
  'West',
  'Congregation',
  'in',
  'Tuscoloosa,',
  'Alabama'],
 ['On',
  'November',
  '11',
  'and',
  'December',
  '12,',
  '2020,',
  'major',
  'storms',
  'swept',
  'through',
  'and',
  'destroyed',
  'parts',
  'of',
  'the',
  'south',
  'and',
  'central',
  'United',
  'States.'],
 ['Big',
  'rain',
  'and',
  'heavy',
  'wind',
  'for',
  'two',
  'days,',
  'along',
  'with',
  'a',
  'lot',
  'of',
  'tornadoes,',
  'caused',
  'serious',
  'damage',
  'to',
  'many',
  'states.'],
 ['Sadly,',
  'Albert',
  'Barnett',
  'was',
  '85',
  'years',
  'old,',
  'and',
  'his',
  'wife,',
  '75',
  '-',
  'year',
  '-',
  'old',
  'Susan',
  'Barnett,',
  'was',
  'killed',
  'by',
  'a',
  'tornado',
  'that',
  'swept',
  'through',
  'their',
  'home.'],
 ['The',
  'United',
  'States',
  'branch',
  'also',
  'reports',
  'that',
  'at',
  'least',
  'four',
  'homes',
  'of',
  '

In [166]:
hyp1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
'ensures', 'that', 'the', 'military', 'always',
'obeys', 'the', 'commands', 'of', 'the', 'party']
ref1a = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
'ensures', 'that', 'the', 'military', 'will', 'forever',
'heed', 'Party', 'commands']
ref1b = ['It', 'is', 'the', 'guiding', 'principle', 'which',
'guarantees', 'the', 'military', 'forces', 'always',
'being', 'under', 'the', 'command', 'of', 'the', 'Party']
ref1c = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
'army', 'always', 'to', 'heed', 'the', 'directions',
'of', 'the', 'party']
hyp2 = ['he', 'read', 'the', 'book', 'because', 'he', 'was',
'interested', 'in', 'world', 'history']
ref2a = ['he', 'was', 'interested', 'in', 'world', 'history',
'because', 'he', 'read', 'the', 'book']
list_of_references = [[ref1a], [ref2a]]
hypotheses = [hyp1, hyp2]
corpus_bleu(list_of_references, hypotheses)

0.5314082679468591

In [171]:
tokenized_sentences2 = [[sentence.split()] for sentence in sentences]
corpus_bleu(tokenized_sentences2, tokenized_translated_en)

0.15188227204547033

In [175]:
tokenized_translated_en

[['Albert',
  'Barnett',
  'and',
  'Susan',
  'Barnett,',
  'members',
  'of',
  'the',
  'West',
  'Congregation',
  'in',
  'Tuscoloosa,',
  'Alabama'],
 ['On',
  'November',
  '11',
  'and',
  'December',
  '12,',
  '2020,',
  'major',
  'storms',
  'swept',
  'through',
  'and',
  'destroyed',
  'parts',
  'of',
  'the',
  'south',
  'and',
  'central',
  'United',
  'States.'],
 ['Big',
  'rain',
  'and',
  'heavy',
  'wind',
  'for',
  'two',
  'days,',
  'along',
  'with',
  'a',
  'lot',
  'of',
  'tornadoes,',
  'caused',
  'serious',
  'damage',
  'to',
  'many',
  'states.'],
 ['Sadly,',
  'Albert',
  'Barnett',
  'was',
  '85',
  'years',
  'old,',
  'and',
  'his',
  'wife,',
  '75',
  '-',
  'year',
  '-',
  'old',
  'Susan',
  'Barnett,',
  'was',
  'killed',
  'by',
  'a',
  'tornado',
  'that',
  'swept',
  'through',
  'their',
  'home.'],
 ['The',
  'United',
  'States',
  'branch',
  'also',
  'reports',
  'that',
  'at',
  'least',
  'four',
  'homes',
  'of',
  '